<a href="https://colab.research.google.com/github/cicyfan/python-vis/blob/master/nyc-vehicle-collision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NYC Vehicle Collisions

Resources:

[Vehicle Collision Data](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95)

[Socrata API Doc](https://dev.socrata.com/foundry/data.cityofnewyork.us/h9gi-nx95)

[zipcode to neighborhood](https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm)


In [1]:
!pip install sodapy

In [2]:
import pandas as pd
import altair as alt
from sodapy import Socrata
import datetime as dt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [0]:
client = Socrata("data.cityofnewyork.us", "FSftCRf0oX5vPMmtWkvtr6DXD")
results = client.get("h9gi-nx95", limit=2000000)

In [4]:
data_df = pd.DataFrame.from_records(results)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674306 entries, 0 to 1674305
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   crash_date                     1674306 non-null  object
 1   crash_time                     1674306 non-null  object
 2   latitude                       1472452 non-null  object
 3   longitude                      1472452 non-null  object
 4   location                       1472452 non-null  object
 5   on_street_name                 1342920 non-null  object
 6   number_of_persons_injured      1674289 non-null  object
 7   number_of_persons_killed       1674275 non-null  object
 8   number_of_pedestrians_injured  1674306 non-null  object
 9   number_of_pedestrians_killed   1674306 non-null  object
 10  number_of_cyclist_injured      1674306 non-null  object
 11  number_of_cyclist_killed       1674306 non-null  object
 12  number_of_motorist_injured  

### Clean Data
Turn all text strings to uppercase to optimize grouping

In [0]:
text_columns = ['on_street_name', 'off_street_name', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'vehicle_type_code1', 'vehicle_type_code2', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5' ]
for column in text_columns:
  data_df[column] = data_df[column].str.upper() 


Clean data: fill all empty cells/NaN with 'MISSING' so it can be differentiated from other reasons such as 'UNSPECIFIED' 

In [0]:
data_df[text_columns] = data_df[text_columns].fillna('MISSING')

Extract datetime, month, day, weekday, weeknumber, hour

In [0]:
data_df["crash_dt"] = pd.to_datetime(data_df['crash_date'])
data_df["year"] = data_df.crash_dt.dt.year
data_df["month"] = data_df.crash_dt.dt.month
data_df["weeknumber"] = data_df.crash_dt.dt.weekofyear
data_df["day"] = data_df.crash_dt.dt.day
data_df["dayofyear"] = data_df.crash_dt.dt.dayofyear
data_df["weekday"] = data_df.crash_dt.dt.weekday
data_df['hour'] = pd.to_datetime(data_df['crash_time'], format='%H:%M').dt.hour

### Top Contributing Factors and Vehicle Types

In [8]:
source = data_df.groupby('contributing_factor_vehicle_1').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('contributing_factor_vehicle_1:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
    #color='borough'
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

In [9]:
source = data_df.groupby('contributing_factor_vehicle_2').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('contributing_factor_vehicle_2:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

In [10]:
source = data_df.groupby('vehicle_type_code1').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('vehicle_type_code1:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

In [11]:
source = data_df.groupby('vehicle_type_code2').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('vehicle_type_code2:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

### Year/Day Heatmap

In [12]:
source = data_df.groupby(['year', 'dayofyear']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('dayofyear:O', title='Day of Year', axis=alt.Axis(labelAngle=-0, tickCount=30)),
    alt.Y('year:O', title='Year', axis=alt.Axis(labelAngle=-0)),
    alt.Color('counts', bin=alt.Bin(extent=[min, max], step=100), 
        scale=alt.Scale(scheme='yellowgreenblue'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=800
)


alt.Chart(...)

### Month/Hour Heatmap

In [13]:
source = data_df.groupby(['month', 'hour']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('hour:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('month:O'),
    alt.Color('counts', bin=alt.Bin(extent=[min, max], step=1000), 
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=600,
    title='Month By Hour '
)


alt.Chart(...)

### Weekday/Hour Heatmap

In [14]:
source = data_df.groupby(['weekday', 'hour']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('hour:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('weekday:O'),
    alt.Color('counts', bin=alt.Bin(extent=[min, max], step=2000), 
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=600,
    title='Weekday By Hour '
)



alt.Chart(...)

In [15]:
source = data_df.groupby(['borough', 'zip_code', 'year']).size().reset_index(name='counts')
source = source.sort_values('counts', ascending=False).groupby(['borough', 'year']).head(1)
base = alt.Chart(source).mark_line().encode(
    x=alt.X('year:O', title='Year',axis=alt.Axis(labelAngle=-0)),
    y=alt.Y('counts:Q', title="Total Collisions"),
    color='borough:N',
).properties(
    height=400,
    width=700,
    title='Most dangerous zipcode in each borough over the years'
)

texts = base.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='zip_code:Q'
)
alt.layer(base, texts, data=source)

alt.LayerChart(...)

In [16]:
source = data_df.groupby(['borough', 'zip_code', 'year']).size().reset_index(name='counts')
source = source.sort_values('counts', ascending=False).groupby(['borough', 'year']).head(1)
base = alt.Chart(source).mark_area().encode(
    x=alt.X('year:O', title='Year', axis=alt.Axis(labelAngle=-0)),
    y=alt.Y('counts:Q', title="Total Collisions"),
    color='zip_code:N',
).properties(
    height=200,
    width=300,
)

texts = base.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='zip_code:Q'
)

alt.layer(base, texts, data=source).facet(
    column='borough',
    title='Most dangerous zipcode in each borough over the years'
)

alt.FacetChart(...)